In [1]:
# requer: whisper, ollama, torch, transformers (bark), soundfile
import whisper
from ollama import chat
from transformers import BarkModel, AutoProcessor
import torch
import soundfile as sf

# ASR
w = whisper.load_model("base")
waveform, sr = sf.read("input.wav")   # seu áudio
res = w.transcribe(waveform, fp16=False)
user_text = res["text"]

# LLM
resp = chat(model="phi3:instruct", messages=[
    {"role":"system","content":"You are helpful."},
    {"role":"user","content":user_text},
])
reply_text = resp["message"]["content"]

# TTS (Bark)
device = "cuda" if torch.cuda.is_available() else "cpu"
bark = BarkModel.from_pretrained("suno/bark-small").to(device)
proc = AutoProcessor.from_pretrained("suno/bark")
inputs = proc(reply_text, return_tensors="pt").to(device)
with torch.no_grad():
    audio_tensor = bark.generate(**inputs)   # shape (1, N)
audio = audio_tensor[0].cpu().numpy()
sf.write("reply.wav", audio, 24000)


ModuleNotFoundError: No module named 'soundfile'